In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from vmdpy import VMD  
from scipy.fftpack import fft
import os
import json

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import platform
system_name = platform.system()

In [10]:
tt_flag = "test"
if tt_flag == "train":
    data_path = "../data/train/"
elif tt_flag == "test":
    data_path = "../data/test/"
    
metallographic_path = data_path + "metallographic"
friction_directory = data_path + "friction" 

In [11]:
grade_num_list = ["1cr", "2cr", "3cr", "4cr"]
process_num_list = ["process_01", "process_02", "process_03", "process_04", "process_05", "process_06", "process_07", "process_08",
                    "process_09", "process_10", "process_11", "process_12", "process_13", "process_14", "process_15", "process_16", "process_17", "process_18"]

grade_dict = {'1cr': ['0.08', '1.00', '1.00', '0.04', '0.03', '0.06', '11.50', '86.29'],
              '2cr': ['0.16', '1.00', '1.00', '0.04', '0.03', '0.06', '13.00', '84.71'],
              '3cr': ['0.26', '1.00', '1.00', '0.04', '0.03', '0.06', '12.00', '85.61'],
              '4cr': ['0.36', '0.60', '0.80', '1.04', '0.03', '0.06', '13.00', '84.11']}
process_dict = {'process_01': ['1050', '1000', '160'],
                'process_02': ['1050', '1000', '200'],
                'process_03': ['1050', '1000', '240'],
                'process_04': ['1050', '1020', '160'],
                'process_05': ['1050', '1020', '200'],
                'process_06': ['1050', '1020', '240'],
                'process_07': ['1050', '1040', '160'],
                'process_08': ['1050', '1040', '200'],
                'process_09': ['1050', '1040', '240'],
                'process_10': ['980', '1000', '160'],
                'process_11': ['980', '1000', '200'],
                'process_12': ['980', '1000', '240'],
                'process_13': ['980', '1020', '160'],
                'process_14': ['980', '1020', '200'],
                'process_15': ['980', '1020', '240'],
                'process_16': ['980', '1040', '160'],
                'process_17': ['980', '1040', '200'],
                'process_18': ['980', '1040', '240']}

In [12]:
surface_image_list = []

def get_image_paths(folder_path):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                image_paths.append(os.path.join(root, file))  
    return image_paths


for grade_num in grade_num_list:
    for process_num in process_num_list:

        pic_surface_path = metallographic_path + "/" + grade_num + "/" + process_num + "/surface" 
        for root, dirs, files in os.walk(pic_surface_path):
            for file in files:
                if file.endswith(('.jpg', '.png', '.jpeg')): 
                    surface_image_list.append(os.path.join(root, file))

print( len(surface_image_list))

100


In [13]:
def list_txt_files(directory):
    back_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                # print(os.path.join(root, file))
                back_list.append(os.path.join(root, file))
    return back_list

file_path_list = list_txt_files(friction_directory)

In [14]:
def do_vmd(file_path):
    
    data = pd.read_csv(file_path, delim_whitespace=True, encoding='ansi')

    time_sec = data['时间(Sec)']
    friction_force = data['摩擦系数']
    
    output_data = pd.DataFrame({
        '时间(Sec)': time_sec[:-10],
        '摩擦系数': friction_force[:-10]
    })
    
    output_file_path = file_path[:-4] + ".csv"
    output_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    
    filename= output_file_path
    f = pd.read_csv(filename,usecols=[1])

    alpha = 7000        # moderate bandwidth constraint  
    tau = 0.            # noise-tolerance (no strict fidelity enforcement)  
    K = 2               # 3 modes  
    DC = 0              # no DC part imposed  
    init = 1            # initialize omegas uniformly  
    tol = 1e-7  
    
    u, u_hat, omega = VMD(f.values[:-10], alpha, tau, K, DC, init, tol) 
    friction_value = f.iloc[:, 0].tolist()

    save_flag = 0 
    imf_list = []
    for i  in range(K):
        a = u[i,:]
        a_rounded = [round(num, 3) for num in a]
        imf_list.append(a_rounded[:570]) 
        if save_flag:
            dataframe = pd.DataFrame({'v{}'.format(i+1):a})
            dataframe.to_csv(file_path[:-4] + "_VMDban-%d.csv"%(i+1),index=False,sep=',')  

    return friction_value[:570]

In [15]:
friction_data_list = []
for pic_surface_path in surface_image_list:
    path_parts = os.path.normpath(pic_surface_path).split(os.sep)
    if system_name == "Windows":
        image_path = "../../datasets" + pic_surface_path[2:]
    elif system_name == "Linux":
        image_path = "/hy-tmp/datasets" + pic_surface_path[2:]
    
    grade_value = grade_dict[path_parts[-4]]
    process_value = process_dict[path_parts[-3]]

    try:
        friction_path = "../../datasets/" + path_parts[1] + "/" + path_parts[2] + "/friction/" + path_parts[4] + "/process"  + path_parts[5][-2:] + ".txt"
        
        friction_list = do_vmd(friction_path)

    except FileNotFoundError:
        pass 
    
    friction_data_list.append({
        "image": image_path,
        "grade": grade_value,
        "process": process_value,
        "friction_list": friction_list
    })



In [16]:
with open(data_path + "/data_friction.json", 'w') as f:
    json.dump(friction_data_list, f, indent=4)